## Final Project Submission

Please fill out:
* Student name: Alex Billinger
* Student pace: full time
* Scheduled project review date/time: TBD (~ 5/11/20)
* Instructor name: Rafael Carrasco
* Blog post URL: https://anbillinger.github.io/data_visualization


### Approach to project
First, we divided the datasets to clean and get basic information. For my cleaning, I primarly filled missing numerical data with the median, and missing categorical data with "missing data" as a new category. I also converted strings representing monetary values into integers. After that, we looked through the data to find ideas for what aspects of data to compare. The questions we decided to work to answer were: 
* What are the most lucrative genres?
* Does having more writers have a significant impact on budget or revenue?
* What is the relationship between budget and revenue?
* Does runtime have an impact on audience enjoyment?
* What regions are most popular for movie releases?
<br><br>
I tackled the questions about genres and number of writers. These questions are examined and answered in different notebooks, and I will explain my methodology in the respective notebook.

In [104]:
import pandas as pd
import seaborn as sns
import numpy as np
from pandasql import sqldf

# Potential points of interest
- ratings/budget/gross relationship
- impact of length of movie
- how will does each genre do?

In [33]:
#convert data sets to pandas dataframes
imdb_basics_df = pd.read_csv('zippedData\imdb.title.basics.csv.gz')
imdb_ratings_df = pd.read_csv('zippedData\imdb.title.ratings.csv.gz')
rt_reviews_df = pd.read_csv('zippedData/rt.reviews.tsv.gz', compression='gzip',error_bad_lines=False, delimiter='\t',encoding='ISO-8859-1')
tmdb_df = pd.read_csv('zippedData/tmdb.movies.csv.gz')
tn_budgets_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

# IMDB Titles Basics
Total rows of data: 146144<br>
Column Names: tconst (primary key), primary_title, original_title, start_year, runtime_minutes, genres
- 31739 NaN in 'runtime_minutes'
- Mean: 86.187247 min, median: approx 87 min, chose median to fill NaN values
- Mean after cleaning: 86.363758
<br><br>
- 5408 NaN in genres
- Mode = Documentary at 32185
- Not many out of total, but replacing with mode seems inaccurate
- Will create a category of "Genre Data Missing"
<br><br>
- 21 NaN in original_title
- Will fill with primary title

In [42]:
#Clean IMDB basics runtime_minutes
imdb_basics_df['runtime_minutes'].fillna(imdb_basics_df['runtime_minutes'].median(),inplace=True)
#Clean IMDB basics genres
imdb_basics_df['genres'].fillna('Genre Data Missing',inplace = True)
#Clean IMDB basics original_title
imdb_basics_df['original_title'].fillna(imdb_basics_df['primary_title'].median(),inplace = True)

print ('IMDB title basics')
print(imdb_basics_df.columns)
print('Number of entries:', len(imdb_basics_df),'\n')
print(imdb_basics_df.isna().sum())

IMDB title basics
Index(['tconst', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres'],
      dtype='object')
Number of entries: 146144 

tconst             0
primary_title      0
original_title     0
start_year         0
runtime_minutes    0
genres             0
dtype: int64


# IMDB Title Ratings
Total Rows of Data: 73856<br>
Column Names: tconst (primary key), averagerating, numvotes<br>
No NaN values

In [43]:
print ('IMDB title ratings')
print(imdb_ratings_df.head())
print('\nNumber of entries:',len(imdb_ratings_df),'\n')
print(imdb_ratings_df.isna().sum())

IMDB title ratings
       tconst  averagerating  numvotes
0  tt10356526            8.3        31
1  tt10384606            8.9       559
2   tt1042974            6.4        20
3   tt1043726            4.2     50352
4   tt1060240            6.5        21

Number of entries: 73856 

tconst           0
averagerating    0
numvotes         0
dtype: int64


# RT Reviews
Number of rows: 54432<br>
Column Names: id, review, rating, fresh, critic, top_critic, publisher, date

- 13517 NaN in 'rating'<br><br>
Did not use this data set in any analysis, so cleaning not completed

In [57]:
print ('RT Reviews')
print(rt_reviews_df.head())
print(rt_reviews_df.columns)
print('\nNumber of entries:',len(rt_reviews_df),'\n')
print(rt_reviews_df.isna().sum())

RT Reviews
   id                                             review rating   fresh  \
0   3  A distinctly gallows take on contemporary fina...    3/5   fresh   
1   3  It's an allegory in search of a meaning that n...    NaN  rotten   
2   3  ... life lived in a bubble in financial dealin...    NaN   fresh   
3   3  Continuing along a line introduced in last yea...    NaN   fresh   
4   3             ... a perverse twist on neorealism...     NaN   fresh   

           critic  top_critic         publisher               date  
0      PJ Nabarro           0   Patrick Nabarro  November 10, 2018  
1  Annalee Newitz           0           io9.com       May 23, 2018  
2    Sean Axmaker           0  Stream on Demand    January 4, 2018  
3   Daniel Kasman           0              MUBI  November 16, 2017  
4             NaN           0      Cinema Scope   October 12, 2017  
Index(['id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher',
       'date'],
      dtype='object')

Number

In [129]:
rt_reviews_df['fresh'].value_counts()

fresh     33035
rotten    21397
Name: fresh, dtype: int64

# TMDB Movies
Rows of data: 26517 <br>
Column Names: genre_ids, id (primary key), original_language, original_title, popularity, release_date, title, vote_average, vote_count<br>
Originally had extra column "Unnamed: 0" which was a duplicate of the index<br>
No NaN values

In [59]:
tmdb_df.drop('Unnamed: 0',axis=1,inplace = True)
print ('TMDB')
print(tmdb_df.head())
print(tmdb_df.columns)
print('\nNumber of entries:',len(tmdb_df),'\n')
print(tmdb_df.isna().sum())

TMDB
             genre_ids     id original_language  \
0      [12, 14, 10751]  12444                en   
1  [14, 12, 16, 10751]  10191                en   
2        [12, 28, 878]  10138                en   
3      [16, 35, 10751]    862                en   
4        [28, 878, 12]  27205                en   

                                 original_title  popularity release_date  \
0  Harry Potter and the Deathly Hallows: Part 1      33.533   2010-11-19   
1                      How to Train Your Dragon      28.734   2010-03-26   
2                                    Iron Man 2      28.515   2010-05-07   
3                                     Toy Story      28.005   1995-11-22   
4                                     Inception      27.920   2010-07-16   

                                          title  vote_average  vote_count  
0  Harry Potter and the Deathly Hallows: Part 1           7.7       10788  
1                      How to Train Your Dragon           7.7        7610  
2  

# TN Movie Budgets
Rows of data: 5782<br>
Column Names: id, release_date, movie, production_budget, domestic_gross, worldwide_gross<br>
No NaNs<br>
Production budget, domestic gross, and worldwide gross converted from strings to numeric values

In [79]:
money_list = ['production_budget','domestic_gross','worldwide_gross']
for ser in money_list:
    tn_budgets_df[ser] = tn_budgets_df[ser].apply(lambda x:x.replace(',',''))
    tn_budgets_df[ser] = tn_budgets_df[ser].apply(lambda x:x.replace('$',''))
    tn_budgets_df[ser] = pd.to_numeric(tn_budgets_df[ser])

print ('TN Movie Budgets')
print(tn_budgets_df.head())
print(tn_budgets_df.columns)
print('\nNumber of entries:',len(tn_budgets_df),'\n')
print(tn_budgets_df.isna().sum())

TN Movie Budgets
   id  release_date                                        movie  \
0   1  Dec 18, 2009                                       Avatar   
1   2  May 20, 2011  Pirates of the Caribbean: On Stranger Tides   
2   3   Jun 7, 2019                                 Dark Phoenix   
3   4   May 1, 2015                      Avengers: Age of Ultron   
4   5  Dec 15, 2017            Star Wars Ep. VIII: The Last Jedi   

   production_budget  domestic_gross  worldwide_gross  
0          425000000       760507625       2776345279  
1          410600000       241063875       1045663875  
2          350000000        42762350        149762350  
3          330600000       459005868       1403013963  
4          317000000       620181382       1316721747  
Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')

Number of entries: 5782 

id                   0
release_date         0
movie                0
production_budge

In [76]:
tn_budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null int32
domestic_gross       5782 non-null int32
worldwide_gross      5782 non-null int64
dtypes: int32(2), int64(2), object(2)
memory usage: 226.0+ KB


## IMDB title crew
- Rows of data: 146144
- NaN in directors: 5727
- NaN in writers: 35883
- if missing *both* will delete row - 141670 remain
- missing only one, info will be filled with "missing data"

In [120]:
imdb_crew_df = pd.read_csv('zippedData/imdb.title.crew.csv.gz')
imdb_crew_df = imdb_crew_df.query('directors == directors | writers == writers')
imdb_crew_df['directors'].fillna('Missing director data',inplace=True)
imdb_crew_df['writers'].fillna('Missing writer data',inplace=True)
imdb_crew_df.head()

,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,Missing director data,"nm0175726,nm1802864"
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


In [121]:
imdb_crew_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141670 entries, 0 to 146143
Data columns (total 3 columns):
tconst       141670 non-null object
directors    141670 non-null object
writers      141670 non-null object
dtypes: object(3)
memory usage: 4.3+ MB


Following cells used to generate CSV file, since the ON OR clause takes a very long time to run, and this data is reusable. Combines TN budget data with IMDB movie basics data, which allows financial analysis for more complex questions. The ON clause compares any budget data where the title is the same to the primary title *or* to the original title.

In [123]:
q = """SELECT tconst, primary_title, original_title, directors, writers, genres
    FROM imdb_basics_df
    JOIN imdb_crew_df
    USING(tconst);"""

imdb_crew_movies = sqldf(q)
imdb_crew_movies.head(10)

,tconst,primary_title,original_title,directors,writers,genres
0,tt0063540,Sunghursh,Sunghursh,nm0712540,"nm0023551,nm1194313,nm0347899,nm1391276","Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,nm0002411,Missing writer data,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,nm0000080,"nm0000080,nm0462648",Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,nm0611531,nm0347899,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,"nm0765384,nm0749914","nm1360635,nm0749914","Comedy,Drama,Fantasy"
5,tt0111414,A Thin Life,A Thin Life,nm0398271,nm0398271,Comedy
6,tt0112502,Bigfoot,Bigfoot,nm6883878,Missing writer data,"Horror,Thriller"
7,tt0137204,Joe Finds Grace,Joe Finds Grace,nm0365480,nm0365480,"Adventure,Animation,Comedy"
8,tt0139613,O Silêncio,O Silêncio,"nm0518037,nm0023406",Missing writer data,"Documentary,History"
9,tt0144449,Nema aviona za Zagreb,Nema aviona za Zagreb,nm0309428,nm2898786,Biography


In [124]:
q = """SELECT movie, genres, directors, writers, production_budget, domestic_gross, worldwide_gross, tconst
    FROM imdb_crew_movies imdb
    JOIN tn_budgets_df tn
    ON (imdb.primary_title = tn.movie OR imdb.original_title = tn.movie);"""

df = sqldf(q)
df.head(10)

,movie,genres,directors,writers,production_budget,domestic_gross,worldwide_gross,tconst
0,Foodfight!,"Action,Animation,Comedy",nm0440415,"nm0440415,nm0923312,nm0295165,nm0841854,nm0220297",45000000,0,73706,tt0249516
1,Mortal Kombat,"Action,Adventure,Fantasy",nm2585406,nm3859643,20000000,70433227,122133227,tt0293429
2,The Overnight,Genre Data Missing,nm1208371,"nm1170333,nm1208371",200000,1109808,1165996,tt0326592
3,On the Road,"Adventure,Drama,Romance",nm0758574,"nm0449616,nm1433580",25000000,720828,9313302,tt0337692
4,The Secret Life of Walter Mitty,"Adventure,Comedy,Drama",nm0001774,"nm0175726,nm0862122",91000000,58236838,187861183,tt0359950
5,A Walk Among the Tombstones,"Action,Crime,Drama",nm0291082,"nm0088747,nm0291082",28000000,26017685,62108587,tt0365907
6,Jurassic World,"Action,Adventure,Sci-Fi",nm1119880,"nm0415425,nm0798646,nm1119880,nm2081046,nm0000341",215000000,652270625,1648854864,tt0369610
7,The Rum Diary,"Comedy,Drama",nm0732430,"nm0732430,nm0860219",45000000,13109815,21544732,tt0376136
8,The Three Stooges,"Comedy,Family","nm0268380,nm0268370","nm0148808,nm0268370,nm0268380",30000000,44338224,54052249,tt0383010
9,Anderson's Cross,"Comedy,Drama,Romance",nm1516329,nm1516329,300000,0,0,tt0393049


In [126]:
#Saves the previous join query to a new csv file, to be easily imported to other notebooks
df.to_csv('TN_budget_and_IMDB.csv',index=False)